In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import tensorflow as tf
import timeit
import os
from six.moves import cPickle
from text_utils import TextLoader
from tensorflow.contrib import rnn
from char_rnn_model import Model
import numpy as np

# Define directories, hyperparameter

In [2]:
# data locations, etc
data_dir = '.'

# training
num_epochs = 5
batch_size = 512

# network architecture
num_neurons = 256
seq_length = 50
vocab_size = 67
num_layers = 2 
dropout = False
learning_rate = 0.001


# Load data using TextLoader object

In [3]:
loader = TextLoader('.', 20, 50)
#loader.preprocess_data()

# Create your model object

In [4]:
my_model2 = Model(num_neurons, seq_length, vocab_size, num_layers=3, lr=0.001, use_dropout=True, use_gpu=True)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


## Save

In [5]:
saver = tf.train.Saver()

# Training

In [7]:
init = tf.global_variables_initializer()
i = 0
with tf.Session() as sess:
    sess.run(init)
    for epochs in range(num_epochs):
        print("Training for Epoch Number: \t" + str(epochs))
        for num in range(5):
            print("Loading Data...")
            x_train = loader.load_train_data(num)
            print("Batching Data on:\t " + str(num))
            batcher = loader.training_batch(batch_size)
            print("Beginning Training:")
            for data, labels in batcher:
                i+=1
                f_dict = {my_model2.X:data, my_model2.Y:labels}
                sess.run(my_model2.trainer, feed_dict=f_dict)
                if i % 50 ==0:
                    c_loss = sess.run(my_model2.loss, feed_dict=f_dict)
                    print(c_loss)
        save_path = saver.save(sess, "./tmp/model" + str(epochs) + str(num) + ".ckpt")
        print("Model saved in path: %s" % save_path)


Training for Epoch Number: 	0
Loading Data...
Batching Data on:	 0
Beginning Training:
3.3462667
3.1526585
3.0249832
2.969503
2.755227
2.7415261
2.527471
2.612715
2.3154497
2.24702
2.3499396
2.307338
2.258669
2.203935
2.212308
1.9840947
2.2151518
2.2021444
2.119788
2.3040833
1.9676616
1.9738908
1.9203806
1.8673192
1.794451
1.7448632
1.6377819
1.8404398
1.9517837
2.0525582
1.8275998
1.6224189
1.7911966
Loading Data...
Batching Data on:	 1
Beginning Training:
1.6776607
1.8076684
1.4822129


KeyboardInterrupt: 

# Generating Text